In [1]:
import os
import json
import traceback
import pandas as pd 


In [5]:
from langchain_huggingface import ChatHuggingFace,HuggingFaceEndpoint
from dotenv import load_dotenv
load_dotenv()

key= os.getenv("Hugging_face_api")
print(key)

hf_KAIOZYLvBpNLfBqEvgsWhRDcsFDegfLQHDhf_KAIOZYLvBpNLfBqEvgsWhRDcsFDegfLQHD


In [6]:
llm= HuggingFaceEndpoint(
    repo_id="deepseek-ai/DeepSeek-R1",
    max_length=128,
    temperature=0.5,
    huggingfacehub_api_token=key,
    provider="fireworks"
)

WARNING! max_length is not default parameter.
                    max_length was transferred to model_kwargs.
                    Please make sure that max_length is what you intended.
